In [24]:
# numerical fun with Brady, Georgia and Markus USING JULIA
# insert comments and notes in this cell

In [52]:
using Pkg
using PyPlot
using Plots
using Plotly
using DelimitedFiles

In [69]:
# define parameters

# We solve the PDE on $[0,1]^2 \times [0,T]$

N = 30 + 1; # we assume M = N and the index for our spatial points goes 1,...,N, so 2,...,N-1 are the inner spatial points
h = 1/(N-1); # spatial step size

T = 1;
N_time = 10000 + 1; # the index for our time points goes 1,...,N_time, so 2,...,N_time-1 are the inner time points
tau = T/(N_time - 1); # temporal step size

beta = 0; # we assume beta to be positive
gamma = 1; # we assume gamma to be positive

println("tau = ", tau)
println("h = ", h)


tau = 0.0001
h = 0.03333333333333333


In [70]:
# define initial and coefficient functions for PDE

function alpha(x,y)
    """alpha must be a continuous and strictly positive function on Omega = [0,1]^2 """
    
    #val = (x-1)^2 + (y-1)^2 + 1
    
    val = 1
    
    return val
    
end

function v_0(x,y)
    """initial function for v at t = 0. we assume that v_0 is C^1 on Omega = [0,1]^2"""
    
    val = (x-1/2)^2 + (y-1/2)^2
    
    #val = 1
    
    return val
end
    

v_0 (generic function with 1 method)

In [71]:
# define auxiliary functions

function cart_coord(m, n, t; h = h, tau = tau)
    """Get cartesian coordinates from indices"""
    
    x = h*(m-1)
    y = h*(n-1)
    tt = tau*(t-1)
    
    return (x,y,tt)
end

function plot_sol(S; h=h)
    x = 0:h:1
    y = x'
    
    z = S
    surf(x,y,z)
end

plot_sol (generic function with 1 method)

In [72]:
# define main function


function solve_pde(;N = N, T = T, N_time = N_time, beta = beta, gamma = gamma, alpha = alpha, v_0 = v_0)
    
    # define spatial and temporal stepsize
    h = 1/(N-1)
    tau = T/(N_time - 1)
    
    # define list, will be filled with N_times matrices, each matrix corresponding to one spatial slice at constant t
    pde_solution = []
    
        # compute solutions by stepping through time
    for t = 1:N_time


        # compute initial values at t = 0 via v_0
        if t == 1
            
            S_current = zeros(N,N)
            
            for m = 1:N, n = 1:N
                (x,y,tt) = cart_coord(m, n, t; h = h, tau = tau)              
                S_current[m,n] = v_0(x,y)
            end
            
            # store current solution
            push!(pde_solution, S_current)
        
            
        else
            
            
            # compute inner spatial points for t = 2
            
            S_current = zeros(N,N)
            S_old = pde_solution[t-1]
            
            for m = 2:N-1, n = 2:N-1
                    
                (x,y,tt) = cart_coord(m, n, t; h = h, tau = tau)
                (x_plus,y_plus,tt) = cart_coord(m + 1, n + 1, t; h = h, tau = tau)
                (x_minus, y_minus, tt) = cart_coord(m - 1, n - 1, t; h = h, tau = tau)
                    
                A = alpha(x,y)*(S_old[m+1,n] + S_old[m-1,n] - 4*S_old[m,n] + S_old[m,n+1] + S_old[m,n-1])
                    
                B = (1/4)*(alpha(x_plus,y) - alpha(x_minus,y))*(S_old[m+1,n] - S_old[m-1,n])
                    
                C = (1/4)*(alpha(x,y_plus) - alpha(x,y_minus))*(S_old[m,n+1]-S_old[m,n-1])
                    
                bracket = A + B + C
                    
                S_current[m,n] = S_old[m,n] + (tau/(gamma*(h^2))) * bracket
                    
            end
            
            
            # compute boundary values using boundary condition
            
            for n = 2:N-1
                (x,y,tt) = cart_coord(0, n, t; h = h, tau = tau)
                S_current[1,n] = -(S_current[3,n] - 4*S_current[2,n])/(3 + (2*h*beta/alpha(0,y)))
                S_current[N,n] = (4*S_current[N-1,n] - S_current[N-2,n])/(3 + (2*h*beta/alpha(1,y)))
            end
            
            for m  = 2:N-1
                (x,y,tt) = cart_coord(m, 0, t; h = h, tau = tau)
                S_current[m,1] = -(S_current[m,3] - 4*S_current[m,2])/(3 + (2*h*beta/alpha(x,0)))
                S_current[m,N] = (4*S_current[m,N-1] - S_current[m,N-2])/(3 + (2*h*beta/alpha(x,1)))
            end
            
            
            # compute values at the four corners averaging their three neighbours
            
            S_current[1,1] = (S_current[1,2] + S_current[2,1] + S_current[2,2])/3
            S_current[1, N] = (S_current[2,N] + S_current[1,N-1] + S_current[2,N-1])/3
            S_current[N,1] = (S_current[N-1,1] + S_current[N,2] + S_current[N-1,2])/3
            S_current[N,N] = (S_current[N-1,N] + S_current[N,N-1] + S_current[N-1,N-1])/3         
            
            #store current solution
            push!(pde_solution, S_current)
            
            
            
            
        end
    end

    return pde_solution

end

solve_pde (generic function with 1 method)

In [73]:
# Compile

pde_solution = solve_pde(N = 10, T = T, N_time = 10, beta = beta, gamma = gamma, alpha = alpha, v_0 = v_0);



In [74]:
pde_solution = solve_pde(N = N, T = T, N_time = N_time, beta = beta, gamma = gamma, alpha = alpha, v_0 = v_0);

In [75]:
for t=1:N_time
    if mod(t,100) == 0
        println(t)
        println(pde_solution[t])
        #writedlm( string("pde_solution",t,".csv"), pde_solution[t] , ',')
    end
end

100


[0.308768800106826 0.3094037388346129 0.3036892902845306 0.29462912661032953 0.28292541908623153 0.2694182466587866 0.254979603313435 0.2404233439866769 0.22644466824854967 0.21359264946174447 0.2022708719120592 0.1927566206646262 0.18522845544324174 0.17979412987410318 0.17651397222089557 0.17541764656867778 0.17651397222089557 0.17979412987410318 0.18522845544324174 0.1927566206646262 0.2022708719120592 0.21359264946174447 0.2264446682485496 0.2404233439866769 0.254979603313435 0.2694182466587866 0.2829254190862316 0.29462912661032953 0.3036892902845307 0.3094037388346129 0.308768800106826; 0.3094037388346129 0.30749892265125217 0.30178447410116993 0.29272431042696884 0.28102060290287084 0.26751343047542586 0.25307478713007425 0.23851852780331617 0.22453985206518895 0.21168783327838375 0.20036605572869848 0.19085180448126549 0.18332363925988102 0.17788931369074248 0.17460915603753485 0.17351283038531706 0.17460915603753485 0.17788931369074248 0.18332363925988102 0.19085180448126549 0

0.14703451063693582 0.1335273382094908 0.11908869486413919 0.1045324355373811 0.09055375979925388 0.0777017410124487 0.06637996346276342 0.056865712215330454 0.04933754699394594 0.043903221424807425 0.0406230637715998 0.039526738119382024 0.0406230637715998 0.043903221424807425 0.04933754699394594 0.05686571221533045 0.06637996346276341 0.0777017410124487 0.09055375979925388 0.1045324355373811 0.11908869486413919 0.1335273382094908 0.14703451063693582 0.15873821816103384 0.16779838183523493 0.1735128303853171 0.17541764656867778; 0.1765139722208956 0.17460915603753488 0.1688947074874527 0.15983454381325163 0.1481308362891536 0.13462366386170857 0.12018502051635696 0.10562876118959888 0.09165008545147166 0.07879806666466647 0.06747628911498119 0.05796203786754822 0.05043387264616372 0.0449995470770252 0.04171938942381757 0.0406230637715998 0.04171938942381757 0.04499954707702519 0.05043387264616371 0.057962037867548215 0.06747628911498119 0.07879806666466647 0.09165008545147166 0.105628

 0.26179898192534373 0.27530615435278866 0.2870098618768867 0.2960700255510878 0.30178447410117 0.30368929028453073; 0.3094037388346129 0.30749892265125217 0.30178447410116993 0.29272431042696884 0.28102060290287084 0.26751343047542586 0.25307478713007425 0.23851852780331617 0.22453985206518892 0.21168783327838375 0.20036605572869848 0.19085180448126549 0.18332363925988102 0.17788931369074248 0.17460915603753485 0.17351283038531706 0.17460915603753485 0.17788931369074248 0.18332363925988102 0.19085180448126549 0.20036605572869848 0.21168783327838375 0.22453985206518892 0.23851852780331617 0.25307478713007425 0.26751343047542586 0.28102060290287084 0.29272431042696884 0.30178447410116993 0.30749892265125217 0.3094037388346129; 0.308768800106826 0.3094037388346129 0.3036892902845306 0.29462912661032953 0.28292541908623153 0.2694182466587866 0.254979603313435 0.2404233439866769 0.2264446682485496 0.21359264946174447 0.2022708719120592 0.1927566206646262 0.18522845544324174 0.1797941298741

[0.25714476483127485 0.2574984586989792 0.25431521388964023 0.24918319335863556 0.24237754745894594 0.23425395578016447 0.22522267400077797 0.2157209326731846 0.20618662407634183 0.19703562976341182 0.18864426522514766 0.18133734241285815 0.17538145976539432 0.17098246150711818 0.16828563855584364 0.16737718483284983 0.16828563855584364 0.17098246150711818 0.17538145976539432 0.18133734241285815 0.18864426522514766 0.19703562976341182 0.20618662407634183 0.21572093267318457 0.22522267400077792 0.23425395578016442 0.2423775474589459 0.24918319335863556 0.25431521388964023 0.2574984586989792 0.2571447648312748; 0.2574984586989792 0.25643737709586617 0.2532541322865272 0.24812211175552257 0.24131646585583294 0.23319287417705148 0.22416159239766498 0.2146598510700716 0.20512554247322884 0.19597454816029883 0.1875831836220347 0.18027626080974515 0.17432037816228135 0.16992137990400522 0.16722455695273064 0.16631610322973686 0.16722455695273064 0.16992137990400522 0.17432037816228135 0.18027

...

[0.16529354864226833 0.16529356551457955 0.16529341366377867 0.16529316722520415 0.1652928369847976 0.16529243739627716 0.16529198594853808 0.16529150240021154 0.1652910079148837 0.16529052413482295 0.16529007223375783 0.16528967199016031 0.16528934092159656 0.16528909351803106 0.16528894060764018 0.16528888888289306 0.16528894060764018 0.16528909351803106 0.16528934092159656 0.16528967199016031 0.16529007223375783 0.16529052413482295 0.1652910079148837 0.16529150240021154 0.16529198594853808 0.16529243739627716 0.1652928369847976 0.16529316722520415 0.16529341366377867 0.16529356551457955 0.16529354864226833; 0.16529356551457955 0.16529351489764593 0.16529336304684503 0.1652931166082705 0.16529278636786396 0.16529238677934352 0.16529193533160444 0.16529145178327792 0.16529095729795004 0.1652904735178893 0.16529002161682418 0.16528962137322667 0.16528929030466294 0.16528904290109742 0.16528888999070654 0.16528883826595941 0.16528888999070654 0.16528904290109742 0.16528929030466294 0.16

 0.1652849447415401 0.16528534498513764 0.16528579688620276 0.1652862806662635 0.1652867751515914 0.1652872586999179 0.16528771014765695 0.16528810973617739 0.16528843997658396 0.16528868641515845 0.16528883826595936 0.16528888888289298; 0.16528894060764016 0.1652888899907065 0.16528873813990558 0.1652884917013311 0.16528816146092454 0.1652877618724041 0.16528731042466505 0.16528682687633856 0.16528633239101062 0.16528584861094991 0.16528539670988476 0.16528499646628725 0.16528466539772352 0.16528441799415794 0.1652842650837671 0.16528421335901994 0.1652842650837671 0.16528441799415794 0.16528466539772352 0.16528499646628725 0.16528539670988476 0.16528584861094991 0.16528633239101062 0.16528682687633856 0.16528731042466505 0.1652877618724041 0.16528816146092454 0.1652884917013311 0.16528873813990558 0.1652888899907065 0.16528894060764016; 0.16528909351803103 0.1652890429010974 0.16528889105029645 0.16528864461172196 0.16528831437131541 0.16528791478279498 0.16528746333505592 0.16528697

16528967199016031 0.16528934092159656 0.16528909351803106 0.16528894060764018 0.16528888888289306 0.16528894060764018 0.16528909351803106 0.16528934092159656 0.16528967199016031 0.16529007223375783 0.16529052413482295 0.1652910079148837 0.16529150240021154 0.16529198594853808 0.16529243739627716 0.1652928369847976 0.16529316722520415 0.16529341366377867 0.16529356551457955 0.16529354864226833]
2800
[0.16529202906636917 0.1652920404365099 0.16529193810524326 0.16529177203189158 0.16529154948503055 0.1652912802049268 0.16529097597723316 0.1652906501171625 0.16529031688671778 0.1652899908704829 0.1652896863372957 0.16528941661573984 0.16528919351078913 0.1652890267871362 0.1652889237418189 0.16528888888484833 0.1652889237418189 0.1652890267871362 0.16528919351078913 0.16528941661573984 0.1652896863372957 0.1652899908704829 0.16529031688671778 0.1652906501171625 0.16529097597723316 0.1652912802049268 0.16529154948503055 0.16529177203189158 0.16529193810524326 0.1652920404365099 0.165292029

Excessive output truncated after 524292 bytes.

0.1652890570691003 0.1652888909957486 0.16528866844888754 0.16528839916878385 0.16528809494109017 0.1652877690810195 0.16528743585057476 0.16528710983433992 0.16528680530115275 0.16528653557959685 0.16528631247464612 0.1652861457509932 0.16528604270567582 0.1652860078487053 0.16528604270567582 0.1652861457509932 0.16528631247464612 0.16528653557959685 0.16528680530115275 0.16528710983433992 0.16528743585057476 0.1652877690810195 0.16528809494109017 0.16528839916878385 0.16528866844888754 0.1652888909957486 0.1652890570691003 0.16528915940036693 0.16528919351078913; 0.1652890267871362 0.16528899267671399 0.1652888903454474 0.16528872427209565 0.16528850172523463 0.16528823244513094 0.16528792821743724 0.1652876023573666 0.16528726912692185 0.165286943110687 0.16528663857749984 0.16528636885594394 0.16528614575099318 0.16528597902734027 0.16528587598202288 0.16528584112505237 0.16528587598202288 0.16528597902734027 0.16528614575099318 0.16528636885594394 0.16528663857749984 0.16528694311